In [ ]:
import pandas as pd

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import Dataset

#### Load model and tokenizer

In [2]:
# https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2

model_name = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### Read dataset

The dataset was collected from several questions and all comments to these questions left by users on the [psycheforum](https://psycheforum.ru/).

In [4]:
data = pd.read_csv('./make_data/quotes.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6573 entries, 0 to 6572
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    6546 non-null   object
dtypes: object(1)
memory usage: 51.5+ KB


In [5]:
quotes = []
for _, row in data.iterrows():
    quotes.append(str(row['text']))

data_len = len(quotes)

# split to train and test 
training = quotes[:int(data_len*0.9)]
validation = quotes[int(data_len*0.9):]

print(len(training), len(validation))

5915 658


#### Tokenize input

In [6]:
tokenizer.pad_token = tokenizer.eos_token

tokenized_val = tokenizer(validation, truncation=True, padding="max_length", max_length=42)
tokenized_train = tokenizer(training, truncation=True, padding="max_length", max_length=42)

# needs to avoid the error in train()
tokenized_val["labels"] = tokenized_val["input_ids"].copy()
tokenized_train["labels"] = tokenized_train["input_ids"].copy()

tokenized_val = Dataset.from_dict(tokenized_val)
tokenized_train = Dataset.from_dict(tokenized_train)

#### Fine-tuning a pretrained model

In [22]:
# Instantiate a Trainer

training_args = TrainingArguments("test_trainer")
trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_train, eval_dataset=tokenized_val
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [23]:
# Fine-tune the model

trainer.train()

***** Running training *****
  Num examples = 5915
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2220


Step,Training Loss
500,3.101400
1000,2.601600
1500,2.289700
2000,1.983800


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2220, training_loss=2.4475227012290612, metrics={'train_runtime': 2372.2016, 'train_samples_per_second': 7.48, 'train_steps_per_second': 0.936, 'total_flos': 380348317440000.0, 'train_loss': 2.4475227012290612, 'epoch': 3.0})

#### NLG samples

In [81]:
prompt = "Но меня не отпускает мысль"
inputs = tokenizer.encode(prompt, return_tensors="pt")
outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.95, top_k=60)
generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Но меня не отпускает мысль, что вот так со мной нельзя, как я считаю, нельзя. Нельзя меня оскорблять обзывать и унижать. Нельзя кричать и ругаться. Нельзя обзывать. Нельзя ругаться. Нельзя оскорблять. Нельзя ничего говорить.. не может стабапки  следует из "да ни за неимением слабой депрессии "да". ( это я шмотать- не буду) я буду)


In [70]:
prompt = "Не знаю хочу ли с ним быть"
inputs = tokenizer.encode(prompt, return_tensors="pt")
outputs = model.generate(inputs, max_length=100, do_sample=True, top_p=0.95, top_k=60)
generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Не знаю хочу ли с ним быть дальше, но по себе знаю, что он мне не враг, и жить хорошо для него тоже не значит плохо что для меня. Мне нравится у него семья, в семье я чувствую себя полноценным и нужным человеком, а не тряпкой.


In [80]:
prompt = "Чем друзья в вашем понимании отличаются"
inputs = tokenizer.encode(prompt, return_tensors="pt")
outputs = model.generate(inputs, max_length=200, do_sample=True, top_p=0.95, top_k=60)
generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Чем друзья в вашем понимании отличаются от родных? У вас муж далеко от дома живет? Что это значит что жена с ним видется только по выходным дням? Или вы жена, с которой только один выходной? Она ездит к мужу по делам, и при этом не в чем не расходует?, как вас это все равно должно взам нужно как в вашем понимании быть "выгоношенной"? следует
